By Tony Cirineo   
**Note:** This paper is being distributed as a preliminary draft.  Proofreading and checking of the calculations has not been performed.  Please excuse any typos and other mistakes.  Also, I’m making an educated guesses about how the RF Front End and Digital Beam Forming network work.  So, keep that in mind as you read this paper.  

# Introduction
The purpose of this study is to identify any potential limitations resulting from a 25 MHz Intermediate Frequency (IF) bandwidth being proposed for the implementation of the Commercial Aircraft Based Instrumentation Telemetry System (CBITS) requirements stated in reference [1].   

It’s not unusual for a vehicle under test to have multiple transmitters in the same band transmitting independent telemetry streams.  The independent data streams from a single vehicle can be captured by one antenna beam.  Although multiple beams can also be used to capture each stream, CBITS allows multiple streams per beam.  For each beam, the received RF energy is processed by the RF Front End and the Digital Beam Forming (DBF) channelizer.  The analysis considers several examples to illustrate how telemetry signals might occupy the IF channel.   

The explanation and calculations are presented in a jupyter notebook using python to perform the calculations.  

# Scope
The analysis presented will consider three modulation types, PCM/FM, SOQPSK-TG and ARTM CPM as defined in reference [2].  PCM/FM and SOQPSK-TG are in common usage at this time on many DoD ranges.  The analysis will consider the case of one telemetry stream within the beam and two telemetry streams within in beam. It is assumed, based on our current understanding of system, that multiple streams can be present within the IF channel. These streams could be from any source within the beam.  CBITS is required to process both polarities for each beam.  It is assumed that only one polarity of the beam would be present in the IF channel and the other polarity would occupy a different IF channel.  

The modulation type BPSK was not analyzed since unfiltered BPSK can be very wide band and is unlikely to be used in modern telemetry systems.  Filtered BPSK is not compatible with class C amplifiers so it is also unlikely to be used in modern telemetry systems where constant envelope modulations are preferred.

# CBITS Requirements
The maximum bit rate for each modulation type is 18 Mbps for PCM/FM, 20 Mbps for SOQPSK and 30 Mbps for ARTM CPM.  These requirements are listed in table 2 of reference [1].  The bit rate and the modulation type determine the shape and bandwidth of the transmitted spectrum.   

The beam bandwidth is required to be large enough to accommodate the data rates for the various modulations listed in Table 1 and IAW paragraph 3.3.1.3.3 of reference [1].  My interpretation of this requirement is that the beam bandwidth shall be sufficiently wide enough to accommodate the equivalent of one maximum bit rate data stream modulated with the least frequency efficient modulation allowed by the specification.  This means that 18 Mbps with PCM/FM as the modulation defines the maximum bandwidth signal that CBITS must process.  Although it’s possible to choose from a wide range of frequency deviations, it will be assumed that optimized PCM/FM is being used.  It is also assumed that the 3dB bandwidth of the IF filter should accommodate at least 99% of the power bandwidth of the modulated signal.    

## Questions to be investigated are:  
* Does the 25 MHz IF bandwidth limit the bit rate to less than what is required in the CBITS specification?  
* Is the Analog to Digital Converter (ADC) sample rate sufficient to accurately digitized the telemetry signal?  
* What is the maximum bit rate and minimum center frequency separation that two identical telemetry signals can have and fit with the IF channel?  

# Definitions
In this paper the following definitions are used.  Additional information can be found in chapter 2 of reference [2].   
* 99% power bandwidth:  The 99% power bandwidth represents the frequency range which contains 99% of the mean RF power.  This bandwidth is an important measure of spectral occupancy.  In this paper the 99% bandwidth is used as the measure of bandwidth for the calculations that follow.  The 99% bandwidth can be calculated from the bit rate and the factors given in Table A-2 of reference [2] and is also called the necessary bandwidth on DD Form 1494.   
* Minimum Frequency Separation:  The minimum frequency separation for between two signals can be  calculated by using the formula A-1 in reference [2].   
* PCM/FM: The PCM/FM modulation referred to in this paper is NRZ PCM/FM and is generated with a multi-pole linear phase premod filter with a 3dB cutoff of 0.7 times the data rate and peak deviation set to 0.35 times the data rate.  See paragraph 3.3.2 of reference [2].  
* SOQPSK: In this paper SOQPSK refers to FQPSK-B, FQPSK-JR or SOQPSK-TG as defined in reference [2].  
* IF Bandwidth:  25 MHz was given as the preliminary CBITS IF bandwidth.  It is assumed that 25 MHz is the 3dB filter bandwidth of the IF filters.   As the CBITS design matures, the IF bandwidth may change to a different value.  
* Nyquist rate: The Nyquist rate is the minimum sample rate for alias-free sampling of a band limited signal.  
* Multi-symbol detectors: In this paper, it is assumed the CBITS will use a multi symbol detector type demodulator.

# Analysis
The analysis will begin by looking that the 99% power bandwidth of PCM/FM, SOQPSK and ARTM CPM at the required maximum bit rates.  The bit rate factors in Table A-2 of reference [2].  The 99% bandwidth is the product of the modulation factor and the bit rate.  Since the ADC anti-aliasing  filter characteristics in the DBF are not known, we can only compare the Nyquist rate to the 99% bandwidth.  

The calculations then continue by using the method described in paragraph A4.a of reference [2].  In reference [2] one signal is considered the desired signal and the other is considered the interferer.  In this paper, both signals are desired and must coexist within the IF bandwidth. Therefore the variable names in the calculations are different.  Also, it is assumed that 99% of the signals power must fit within the 3 dB bandwidth of the IF filter in order to avoid degradation of bit error rate performance.
 
#Examples
The following examples step through some calculations to answer the questions posed above.  In these examples, one or two modulated data streams are present in the IF channel.  I’m assuming the reason there would be two modulated data streams within a single IF channel is if these streams originate from one test vehicle with two transmitters.  


In [1]:
import math  # get math libraries

## Example 1: Single PCM/FM stream in the IF channel
In this example we calculate the 99% bandwidth of a PCM/FM signal at the maximum CBITS bit rate for this modulation type.  

In [2]:
bit_rate = 18 #  bit rate of stream in Mbps
B99_percent = bit_rate * 1.16 # NRZ PCM/FM, from Table A-2 of reference [2].
print('99% bandwidth = {:.2f} MHz'.format(B99_percent))

99% bandwidth = 20.88 MHz


The 99% bandwidth is 20.88 MHz for a 18 Mbps data rate.  20.88 MHz will fit within the IF bandwidth.  

In [3]:
# ADC sampling rate and nyquist rate
ADC_sample_rate = 31.25 # MHz
nyquist_rate = ADC_sample_rate/2
print('Nyquist rate = {:.2f} MHz'.format(nyquist_rate))

Nyquist rate = 15.62 MHz


In [4]:
nyquist_rate/1.16

13.469827586206897

Based on an ADC sample rate of 31.25 MHz, the Nyquist rate is 15.62 MHz.  Since the Nyquist rate is less than the 99% bandwidth, the sampled signal will be aliased.  The maximum alias free data rate is 13.47 Mbps, assuming ideal filters.  This is much less than the maximum allowed by the CBITS specification.  

## Example 2: Single SOQPSK stream in the IF channel
In this example we calculate the 99% bandwidth for an SOQPSK signal at the maximum CBITS bit rate for this modulation type.  

In [5]:
bit_rate = 20 #  bit rate of stream in Mbps
B99_percent = bit_rate * 0.78 # SOQPSK, from Table A-2 of reference [2].
print('99% bandwidth = {:.2f} MHz'.format(B99_percent))

99% bandwidth = 15.60 MHz


The 99 % bandwidth is 15.6 MHz for a 20 Mbps data rate.  15.6 MHz will fit within the IF bandwidth.   Since the Nyquist rate is almost equal to the 99% bandwidth, the sampled signal will likely be aliased since real world filters have finite attenuation roll off rates.  

## Example 3: Single ARTM CPM stream in the IF channel
In this example we calculate the 99% bandwidth for an ARTM CPM signal at the maximum CBITS bit rate for this modulation type.  

In [6]:
bit_rate = 30 #  bit rate of stream in Mbps
B99_percent = bit_rate * 0.56 # ARTM CPM, from Table A-2 of reference [2].
print('99% bandwidth = {:.2f} MHz'.format(B99_percent))

99% bandwidth = 16.80 MHz


The 99 % bandwidth is 16.8 MHz for a 30 Mbps data rate.  16.8 MHz will fit within the IF bandwidth.  Since the Nyquist rate is less than the 99% bandwidth, the sampled signal will be aliased.  The maximum aliased free bit rate is about 27 MHz.  

## Example 4: Dual PCM/FM streams in the IF channel
In this example we calculate the frequency span for two data streams sitting side by side within the IF channel.  Both streams have the same bit rate and modulation type.  First the minimum center to center frequency is calculated using the method describe in reference [2] paragraph A.4a.  Then the 99% bandwidth for each stream is calculated to find the upper and lower bounds of the modulated streams sitting side by side in the IF channel.  Since we know the IF channel is 25 MHz wide at the 3dB points and we want 99% of the power to reside within this bandwidth, 8.62 Mbps was determined to be the maximum bit rate by an iterative process.  

In [7]:
# calculate the 99% bandwidth for each stream using bandwidth 
# factor from Table A-2 of reference [2].
R1 = 8.62 #  bit rate of stream #1 in Mbps
R2 = R1 # bit rate of stream #2, set equal to #1

# calculate the minimum frequency separation between stream #1 and stream #2
a1 = 0.5 #  multi-symbol detectors, see Table A-1 of reference [2]
a2 = 1.2 # for PCM/FM
sep_Fa = a1*R1 + a2*R2
sep_Fb = a1*R2 + a2*R1
if sep_Fa >= sep_Fb: # need to use the larger of the two values.
    sep_F = sep_Fa
else:
    sep_F = sep_Fb
sep_F = math.ceil(sep_F) # frequencies are assigned in 1MHz steps, so round up
print('Center frequency to center frequency = {:.1f} MHz'.format(sep_F))

S1_99_percent_bw = R1*1.16 # 99% power bandwidth for stream #1
S2_99_percent_bw = R2*1.16 # 99% power bandwidth for stream #2

# Calculate the span of the two modulated streams
span = sep_F + S1_99_percent_bw/2 + S2_99_percent_bw/2
print('Frequency span = {:.2f} MHz'.format(span))

Center frequency to center frequency = 15.0 MHz
Frequency span = 25.00 MHz


Since the Nyquist rate is  15.62 MHz, we can solve for the bit rate which will produce an edge to edge span equal to 15.62 MHz.  This bit rate is 5.294 MHz.  

## Example 5: Dual SOQPSK streams in the IF channel
This example is similar to example 4, but the modulation type is changed to SOQPSK.  

In [8]:
# calculate the 99% bandwidth for each stream using bandwidth 
# factor from Table A-2 of reference [2].
R1 = 8.181 #  bit rate of stream #1 in Mbps
R2 = R1 # bit rate of stream #2, set equal to #1

# calculate the minimum frequency separation between stream #1 and stream #2
a1 = 0.45 #  for SOQPSK, see Table A-1 of reference [2]
a2 = 0.65 # for SOQPSK
sep_Fa = a1*R1 + a2*R2
sep_Fb = a1*R2 + a2*R1
if sep_Fa >= sep_Fb: # need to use the larger of the two values.
    sep_F = sep_Fa
else:
    sep_F = sep_Fb
sep_F = math.ceil(sep_F) # frequencies are assigned in 1MHz steps, so round up
print('Center frequency to center frequency = {:.1f} MHz'.format(sep_F))

S1_99_percent_bw = R1*0.78 # 99% power bandwidth for stream #1
S2_99_percent_bw = R2*0.78 # 99% power bandwidth for stream #2

# Calculate the span of the two modulated streams
span = sep_F + S1_99_percent_bw/2 + S2_99_percent_bw/2
print('Frequency span = {:.2f} MHz'.format(span))

Center frequency to center frequency = 9.0 MHz
Frequency span = 15.38 MHz


Since we know the IF channel is 25 MHz wide at the 3dB points and we want 99% of the power to reside within this bandwidth, 12.8 Mbps was determined to be the maximum bit rate by an iterative process.  Since the Nyquist rate is  15.62 MHz, we can solve for the bit rate which will produce an edge to edge span equal to 15.62 MHz.  This bit rate is 8.181 MHz.  

## Example 6: Two transmitters out one antenna
In some systems under test, two transmitters will share the same antenna by combining the power out through a hybrid coupler.  Center frequency separation is chosen to avoid or minimize intermodulation products and interference.  The center frequency separation can be much larger than the minimum values calculated above.  In this case the use of two beams is most likely required, one for each stream.

## Example 7: Frequency Management may assign to any authorized frequency 
Frequency management may assign transmitter frequencies to any authorized frequency in the band.  The frequency spacing is not guaranteed to be at the minimum spacing as calculated above.  Center frequencies can be anywhere in the band, so multiple beams would be required.

# Conclusions and recommendations
Based on the analysis presented above the following conclusions are provided:  
* The ADC sample rate of 31.25 MHz does not support maximum data rates for PCM/FM, SOQPSK and ARTM CPM.  
* The proposed IF bandwidth of 25 MHz is sufficient to meet CBITS requirements.  
  
My recommendation is to allow the CBITS design to move forward with the ADC sample rate of 31.25 MHz.  In practice, as bit rates move higher, more bandwidth efficient modulations are favored.  Therefore, not being able to use the highest bit rates with PCM/FM is not a big limitation, since users will naturally migrate to SOQPSK for these high bit rates.  The same holds true in the 15 to 20 Mbps range, ARTM CPM can be used.  At the very extreme end, a limit somewhere below 27 Mbps will be reached depending on anti-aliasing filter characteristics.  Sometime after PDR, when the CBITS design is mature, have Raytheon request a waiver/deviation for the high end bit rates.  At that time we will know the anti-aliasing filter characteristics.  In the meantime, carry this issue as a risk item and close it when a waiver is granted.  

# iPython Notebook
This paper is available as an iPython notebook and was converted to PDF via LaTeX.  

# References  
[1] SP3859ASY00503 REV B, PERFORMANCE SPECIFICATION, COMMERCIAL AIRCRAFT BASED INSTRUMENTATION, TELEMETRY SYSTEM (CBITS), 19 September 2016  
[2]  IRIG STANDARD 106-17, TELEMETRY STANDARDS  

# Revision
Last updated: 7 May 2018  